In [87]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import cv2
import os
import tensorflow as tf
import json
import crop


from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

### _08: 턱쳐짐(chin_sagging)에 대해 bbox_crop

In [88]:
label_key = "_F_08.json"
annotation_key = "chin_sagging"
image_data, target_data = crop.process_files(label_key, annotation_key)

In [89]:
print(f"Number of images processed: {len(image_data)}")
print(f"Number of targets processed: {len(target_data)}")

Number of images processed: 2574
Number of targets processed: 2574


### 데이터 저장 및 불러오기

In [90]:
# 이미지, 타겟 데이터 저장
with open('chin_sagging_image_data.pkl', 'wb') as f:
    pickle.dump(image_data, f)

with open('chin_sagging_target_data.pkl', 'wb') as f:
    pickle.dump(target_data, f)

In [91]:
# 데이터 로드
with open('chin_sagging_image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)

with open('chin_sagging_target_data.pkl', 'rb') as f:
    target_data = pickle.load(f)

In [92]:
# 데이터 카운트
from collections import Counter

values_list = list(target_data.values())
value_counts = Counter(values_list)

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 0, Count: 1167
Value: 1, Count: 519
Value: 2, Count: 333
Value: 3, Count: 324
Value: 5, Count: 87
Value: 4, Count: 141
Value: 6, Count: 3


### 0, (2,3,4,5,6)으로 모델링

In [93]:
def merge_classes(target_data):
    binary_target_data = {}
    for key, value in target_data.items():
        if value in [2, 3, 4, 5, 6]:
            binary_target_data[key] = '10'
        else:
            binary_target_data[key] = value
    
    return binary_target_data

binary_target_data = merge_classes(target_data)

In [94]:
filtered_target_data = {key: value for key, value in binary_target_data.items() if value in [0, '10']}


values_list = list(filtered_target_data.values())
value_counts = Counter(values_list)

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 0, Count: 1167
Value: 10, Count: 888


In [95]:
common_keys = set(image_data.keys()) & set(filtered_target_data.keys())

# 데이터 정렬
filtered_image_data = {key: image_data[key] for key in common_keys}
filtered_target_data = {key: filtered_target_data[key] for key in common_keys}

# X와 y 생성
X = np.array(list(filtered_image_data.values()))
y = np.array(list(filtered_target_data.values()))

print(f"Length of X: {len(X)}, Length of y: {len(y)}")

Length of X: 2055, Length of y: 2055


In [96]:
class_mapping = {'0': 0, '10': 1}
y = np.array([class_mapping[label] for label in y])
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (1644, 128, 128, 3), y_train shape: (1644,)
X_test shape: (411, 128, 128, 3), y_test shape: (411,)


In [97]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3])),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # 출력층
])

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_39 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_40 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_40 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_41 (MaxPooli  (None, 14, 14, 128)     

In [98]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [99]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    class_weight = class_weights,
    callbacks=[early_stopping]
)

Epoch 1/100
42/42 [==============================] - 10s 213ms/step - loss: 0.6973 - accuracy: 0.5125 - val_loss: 0.6849 - val_accuracy: 0.5745
Epoch 2/100
42/42 [==============================] - 9s 208ms/step - loss: 0.6813 - accuracy: 0.5939 - val_loss: 0.6734 - val_accuracy: 0.5441
Epoch 3/100
42/42 [==============================] - 9s 224ms/step - loss: 0.6785 - accuracy: 0.5825 - val_loss: 0.6673 - val_accuracy: 0.6383
Epoch 4/100
42/42 [==============================] - 9s 221ms/step - loss: 0.6530 - accuracy: 0.6175 - val_loss: 0.6462 - val_accuracy: 0.6322
Epoch 5/100
42/42 [==============================] - 10s 228ms/step - loss: 0.6071 - accuracy: 0.6928 - val_loss: 0.5815 - val_accuracy: 0.6748
Epoch 6/100
42/42 [==============================] - 9s 221ms/step - loss: 0.5415 - accuracy: 0.7262 - val_loss: 0.5553 - val_accuracy: 0.7021
Epoch 7/100
42/42 [==============================] - 9s 208ms/step - loss: 0.5574 - accuracy: 0.7255 - val_loss: 0.4709 - val_accuracy: 0.81

In [100]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

13/13 - 1s - loss: 0.2341 - accuracy: 0.9051 - 599ms/epoch - 46ms/step
Test Loss: 0.23405326902866364
Test Accuracy: 0.9051094651222229


In [101]:
# 예측 수행 (확률 값 반환)
predictions = model.predict(X_test)  # 이진 분류에서는 sigmoid 출력

# 0.5를 기준으로 클래스 결정
predicted_classes = (predictions > 0.5).astype(int).flatten()

print("Predicted Classes: ", predicted_classes)


13/13 [==============================] - 1s 45ms/step
Predicted Classes:  [1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0
 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0
 1 0 1 0 0 1 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 1 1
 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1
 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1
 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1
 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1
 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1
 0 0 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0
 0 1 1 0]
